# Introduction

We show in this Kernel how we can process the data to prepare it for easier processing. Let's check the data files.

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Analysis preparation

## Load packages

In [ ]:
import pandas as pd

## Load the data

The datafiles are in TSV format. We will read the files using pandas, just include in the function call the `sep` (tab separator data).
We demonstrate first how to read and process the Annual data.

In [ ]:
annually_df = pd.read_csv("/kaggle/input/life-expectancy-at-birth-in-europe/tps00208.tsv", sep='\t')

Let's glimpse the data columns.

In [ ]:
print(list(annually_df.columns))

The first column is a composed one, containing 4 different information (the unit for temporal data, sex, type of statistical info, the geography). The next columns are the year value, from first (2009) to last (2019).

# Data pre-processing

We start by defining two working lists.

In [ ]:
pivot_data_col = annually_df.columns[0]
time_columns = annually_df.columns[1:]

Then, we split from `pivot_data_col` the 3 separate fields:
* unit (YR only);
* sex (F/M);
* statistical indicator;
* geography.

In [ ]:
annually_df['unit']     = annually_df[pivot_data_col].apply(lambda x: x.split(",")[0])
annually_df['sex'] = annually_df[pivot_data_col].apply(lambda x: x.split(",")[1])
annually_df['statinfo'] = annually_df[pivot_data_col].apply(lambda x: x.split(",")[2])
annually_df['geography'] = annually_df[pivot_data_col].apply(lambda x: x.split(",")[3])

We select now only the new columns resulted from splitting the `pivot_data_col` and the time columns.

In [ ]:
selected_columns = list(['unit', 'sex', 'statinfo', 'geography']) +  list(time_columns)
annually_df = annually_df[selected_columns]

Next, we pivot the time columns using `melt` operation in pandas.  
We also make sure we transform `date` to be an integer (here is a year data).  
We set `value` to be a float, after we replace ": " (for N/A) with `NAN`.

In [ ]:
annually_tr_df = annually_df.melt(id_vars=['unit', 'sex', 'statinfo', 'geography'], 
        var_name="date", 
        value_name="value")
annually_tr_df['date'] = annually_tr_df['date'].apply(lambda x: int(x))
annually_tr_df['value'] = annually_tr_df['value'].apply(lambda x: str(x).replace(": ", "NAN"))
annually_tr_df['value'] = annually_tr_df['value'].apply(lambda x: float(x))

Let's inspect the result.

In [ ]:
print(f"Transformed data shape: {annually_tr_df.shape} (rows/columns)")
annually_tr_df.head()

In [ ]:
annually_tr_df.tail()

# A very preliminary exploratory data analysis

This would be a very short exploratory data analysis. The role of this Kernel is just to show how we can prepare the annual data for analysis and we already did this.

In [ ]:
import pandas_profiling
pandas_profiling.ProfileReport(annually_tr_df)

# Save reformated dataset

Let's now save the dataset in *.csv format.

In [ ]:
annually_tr_df.to_csv("life_expectancy_at_birth.csv", index=False)